# Pre-processing

Checando manualmente los datos hay muchos que tienen errorer ortográficos, lo que es de esperar pues la mayoría vienen de Twitter. Sería conveniente aplicar un corrector:

-  [contextualSpellCheck](https://spacy.io/universe/project/contextualSpellCheck), de SpaCy, se ve bien ya que usa Bert, pero cambia demasiado los nombres propios y otras entidades por sus respectivos pronombres. Además, borra ciertas groserias y otras palabras que pueden ser importantes para esta tarea.
- [Autocorrect](https://github.com/fsondej/autocorrect) se ve bien aunque es más sencillo, si no reconoce algo lo deja como está, es probable que se me vayan más errores ortográficos con esta librería. Sin embargo, también borra ciertas groserias y otras palabras que pueden ser importantes para esta tarea. 
- [pyspellchecker](https://github.com/barrust/pyspellchecker) también se ve bien. A diferencia de las anteriores, deja las groserias y otras palabras que pueden ser importantes para esta tarea intactas, de modo que me quedaré con este.

El inconveniente es que es un poco tardado hacer este spell-check.

In [1]:
import pandas as pd
import re
from tqdm import tqdm, trange

from utils import preprocessing

In [2]:
text = "Bitches be begging me to follw them just to give me a reason not to follw them. Lol"
%time preprocessing.spell_check(text, language='en')

CPU times: user 572 µs, sys: 781 µs, total: 1.35 ms
Wall time: 1.36 ms


'Bitches be begging me to follow them just to give me a reason not to follow them . Lol'

In [9]:
text = "Las putass me ruegan que las sigaa solo para darme una razón para no seguirlass. Lol"
preprocessing.spell_check(text, language='es')

'Las putas me ruegan que las siga solo para darme una razón para no seguirlas . Lol'

## Train

In [5]:
train = pd.read_csv('../../Data/EXIST2021_training.tsv', sep='\t')
train.head()

,test_case,id,source,language,text,task1,task2
0,EXIST2021,1,twitter,en,"She calls herself ""anti-feminazi"" how about sh...",sexist,ideological-inequality
1,EXIST2021,2,twitter,en,"Now, back to these women, the brave and the be...",non-sexist,non-sexist
2,EXIST2021,3,twitter,en,"@CurvyBandida @Xalynne_B Wow, your skirt is ve...",sexist,objectification
3,EXIST2021,4,twitter,en,@AurelieGuiboud Incredible! Beautiful!But I l...,non-sexist,non-sexist
4,EXIST2021,5,twitter,en,i find it extremely hard to believe that kelly...,non-sexist,non-sexist


In [5]:
corrected_text = []

for value in tqdm(train[['language', 'text']].values):
    language, text = value
    
    corrected_text.append(preprocessing.spell_check(text, language=language))

100%|██████████| 6977/6977 [1:02:53<00:00,  1.85it/s]


In [6]:
train['text'] = corrected_text
train.to_csv('../../Data/EXIST2021_training_spell_checked.csv', index=False)

## Test

In [6]:
test = pd.read_csv('../../Data/EXIST2021_test.tsv', sep='\t')
test.head()

,test_case,id,source,language,text,task1,task2
0,EXIST2021,6978,gab,en,Pennsylvania State Rep horrifies with opening ...,non-sexist,non-sexist
1,EXIST2021,6979,twitter,en,"@iilovegrapes He sounds like as ass, and very ...",non-sexist,non-sexist
2,EXIST2021,6980,twitter,en,"@averyangryskel1 @4ARealistParty LOL! ""This be...",sexist,ideological-inequality
3,EXIST2021,6981,twitter,en,@WanderOrange @stalliontwink Rights?I mean yea...,sexist,ideological-inequality
4,EXIST2021,6982,twitter,en,the jack manifold appreciation i’m seeing is o...,non-sexist,non-sexist


In [7]:
corrected_text = []

for value in tqdm(test[['language', 'text']].values):
    language, text = value
    
    corrected_text.append(preprocessing.spell_check(text, language=language))

100%|██████████| 4368/4368 [36:44<00:00,  1.98it/s]  


In [8]:
test['text'] = corrected_text
test.to_csv('../../Data/EXIST2021_test_spell_checked.csv', index=False)